In [0]:
# dbutils.widgets.removeAll()

In [0]:
import requests

# Define the endpoint names
endpoints = [
    "mmt_scimilarity_gene_order",
    "mmt_scimilarity_get_embedding",
    "mmt_scimilarity_search_nearest"
]

# Define the Databricks instance and token
databricks_instance = "adb-830292400663869.9.azuredatabricks.net"
DATABRICKS_TOKEN = dbutils.secrets.get("mmt", "databricks_token")

# Function to start an endpoint
def start_endpoint(databricks_instance, endpoint_name, token):
    url = f"https://{databricks_instance}/api/2.0/serving-endpoints/{endpoint_name}/config:start"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    response = requests.post(url, headers=headers)
    if response.status_code == 200:
        print(f"Successfully started endpoint: {endpoint_name}")
    else:
        print(f"Failed to start endpoint: {endpoint_name}, Status Code: {response.status_code}, Response: {response.text}")


# Function to stop an endpoint
def stop_endpoint(databricks_instance, endpoint_name, token):
    url = f"https://{databricks_instance}/api/2.0/serving-endpoints/{endpoint_name}/config:stop"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    response = requests.post(url, headers=headers)
    if response.status_code == 200:
        print(f"Successfully stopped endpoint: {endpoint_name}")
    else:
        print(f"Failed to stop endpoint: {endpoint_name}, Status Code: {response.status_code}, Response: {response.text}")


# Function to check the status of an endpoint
def check_endpoint_status(databricks_instance, endpoint_name, token):
    url = f"https://{databricks_instance}/api/2.0/serving-endpoints/{endpoint_name}"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        status = response.json().get("state", {}).get("ready", "Unknown")
        print(f"Endpoint: {endpoint_name}, Status: {status}")
        return status
    else:
        print(f"Failed to check status of endpoint: {endpoint_name}, Status Code: {response.status_code}, Response: {response.text}")
        return "Unknown"
    

## Example USAGE
# Loop through the endpoints and start/stop them OR check their status
# for endpoint in endpoints:
#     start_endpoint(databricks_instance, endpoint, DATABRICKS_TOKEN)
    # stop_endpoint(databricks_instance, endpoint, DATABRICKS_TOKEN)
    # check_endpoint_status(databricks_instance, endpoint, DATABRICKS_TOKEN)

In [0]:
# endpoint_status = []

for endpoint in endpoints:
    start_endpoint(databricks_instance, endpoint, DATABRICKS_TOKEN)
    # stop_endpoint(databricks_instance, endpoint, DATABRICKS_TOKEN)
    
#     endpoint_status.append(check_endpoint_status(databricks_instance, endpoint, DATABRICKS_TOKEN))

# endpoint_status

In [0]:
# ~15mins

import time

# Function to wait until all endpoints are ready
def wait_until_endpoints_ready(databricks_instance, endpoints, token, check_interval=60):
    all_ready = False
    while not all_ready:
        all_ready = True
        for endpoint in endpoints:
            status = check_endpoint_status(databricks_instance, endpoint, token)
            if status != "READY":
                all_ready = False
                print(f"Endpoint {endpoint} is not ready yet. Checking again in {check_interval} seconds...")
                break
        if not all_ready:
            time.sleep(check_interval)
    print("All endpoints are ready!")

### 
wait_until_endpoints_ready(databricks_instance, endpoints, DATABRICKS_TOKEN, check_interval=60)